The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release --no-deps --force-reinstall
!pip install numpy>=1.20


  Cloning https://github.com/d2l-ai/d2l-zh (to revision release) to /tmp/pip-req-build-_lk4xe_z
  Running command git clone --filter=blob:none --quiet https://github.com/d2l-ai/d2l-zh /tmp/pip-req-build-_lk4xe_z
  Running command git checkout -b release --track origin/release
  Switched to a new branch 'release'
  Branch 'release' set up to track remote branch 'release' from 'origin'.
  Resolved https://github.com/d2l-ai/d2l-zh to commit 843d3d41dca48d8df65f4b324dd171d8bfe9c067
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for d2l: filename=d2l-2.0.0-py3-none-any.whl size=110800 sha256=3ff5a9b455ec2e85d757075424a95f7912daa342b1903e1436248763b2af2a9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ozgziqk8/wheels/3e/87/36/53b013ece7da539a59fcc56da2d329d7afb08eef9a9a5ae1d1
Successfully built d2l


# 文本预处理
:label:`sec_text_preprocessing`

对于序列数据处理问题，我们在 :numref:`sec_sequence`中
评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，文本是最常见例子之一。
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。


In [2]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [3]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

正在从http://d2l-data.s3-accelerate.amazonaws.com/timemachine.txt下载../data/timemachine.txt...
# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
[**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [4]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [25]:
class Vocab:#@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):#reserved_tokens 开头和结尾torken
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], #lambda x: x[1] 是一个匿名函数，它指定按照每个元组的第二个元素（即计数）进行排序
                                  reverse=True)#排序后的结果
        print(self._token_freqs)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx: #给你一个token我给你一个index这是字典
                self.idx_to_token.append(token) #索引到token: ['<unk>', '<pad>', '<bos>', '<eos>']
                self.token_to_idx[token] = len(self.idx_to_token) - 1
                #{'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3}
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):#还能这么判断是否为List
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line] #[表达式 for 外层变量 in 外层可迭代对象 for 内层变量 in 内层可迭代对象]
    return collections.Counter(tokens)#count

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。


In [26]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('hello', 3), ('world', 2), ('python', 1), ('test', 1)]
[('<unk>', 0), ('hello', 1), ('world', 2), ('python', 3), ('test', 4)]


现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [27]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: hello
索引: 1


IndexError: list index out of range

## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [28]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

[(' ', 29927), ('e', 17838), ('t', 13515), ('a', 11704), ('i', 10138), ('n', 9917), ('o', 9758), ('s', 8486), ('h', 8257), ('r', 7674), ('d', 6337), ('l', 6146), ('m', 4043), ('u', 3805), ('c', 3424), ('f', 3354), ('w', 3225), ('g', 3075), ('y', 2679), ('p', 2427), ('b', 1897), ('v', 1295), ('k', 1087), ('x', 236), ('z', 144), ('j', 97), ('q', 95)]


(170580, 28)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。

## 练习

1. 词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。
1. 在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？


BPE	简单有效，广泛使用	可能产生不直观的分割	通用NLP任务，多语言处理
WordPiece	处理未知词汇效果好	训练计算成本较高	BERT等Transformer模型
SentencePiece	语言无关，无需预处理	需要训练模型	处理混合语言文本，非标准文本


In [29]:
import collections

def count_corpus(tokens):
    """统计词元的频率"""
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens) #
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        print("_token_freqs:",self._token_freqs)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens #['<pad>', '<bos>', '<eos>']
        print("idx_to_token: ",self.idx_to_token)
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        print("token_to_idx:",self.token_to_idx)
        for token, freq in self._token_freqs: #这里只取前两个
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token) #后面加两个_token_freqs的单词
                self.token_to_idx[token] = len(self.idx_to_token) - 1 #给字典加一个index
        print("new_token_to_idx: ",self.token_to_idx)

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

# 测试代码
if __name__ == "__main__":
    # 示例1: 使用一维token列表
    print("=== 示例1: 一维token列表 ===")
    tokens = ["hello", "world", "hello", "python", "world", "hello", "test"]
    vocab = Vocab(tokens, min_freq=0, reserved_tokens=['<pad>', '<bos>', '<eos>'])

    print("词汇表大小:", len(vocab))
    print("索引到token映射:", vocab.idx_to_token)
    print("token到索引映射:", vocab.token_to_idx)
    print("词频统计:", vocab.token_freqs)

    # 测试转换功能
    test_tokens = ["hello", "world", "python", "unknown_word", "<bos>"]
    indices = vocab[test_tokens] #vocab返回是：vocab.token_to_idx 这个字典就是字典里的索引
    print("Tokens to indices:", test_tokens, "→", indices)

    # 测试反向转换
    test_indices = [0, 1, 2, 3, 4, 5]
    tokens_back = vocab.to_tokens(test_indices)
    print("Indices to tokens:", test_indices, "→", tokens_back)

    # 示例2: 使用二维token列表（句子列表）
    print("\n=== 示例2: 二维token列表 ===")
    sentences = [
        ["I", "love", "python"],
        ["Python", "is", "awesome"],
        ["I", "love", "coding"],
        ["Python", "is", "fun"]
    ]

    vocab2 = Vocab(sentences, min_freq=2, reserved_tokens=['<pad>'])

    print("词汇表大小:", len(vocab2))
    print("索引到token映射:", vocab2.idx_to_token)
    print("词频统计:", vocab2.token_freqs)

    # 测试单个token转换
    print("'python'的索引:", vocab2["python"])
    print("'love'的索引:", vocab2["love"])
    print("'unknown'的索引:", vocab2["unknown"])  # 应该返回0 (<unk>)

    # 测试列表转换
    test_sentence = ["I", "love", "python", "and", "coding"]
    indices2 = vocab2[test_sentence]
    print("句子转索引:", test_sentence, "→", indices2)

    # 测试反向转换
    test_indices2 = [0, 1, 2, 3, 4, 5]
    tokens_back2 = vocab2.to_tokens(test_indices2)
    print("索引转token:", test_indices2, "→", tokens_back2)

    # 测试属性访问
    print("未知token的索引:", vocab2.unk)

=== 示例1: 一维token列表 ===
_token_freqs: [('hello', 3), ('world', 2), ('python', 1), ('test', 1)]
idx_to_token:  ['<unk>', '<pad>', '<bos>', '<eos>']
token_to_idx: {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3}
new_token_to_idx:  {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3, 'hello': 4, 'world': 5, 'python': 6, 'test': 7}
词汇表大小: 8
索引到token映射: ['<unk>', '<pad>', '<bos>', '<eos>', 'hello', 'world', 'python', 'test']
token到索引映射: {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3, 'hello': 4, 'world': 5, 'python': 6, 'test': 7}
词频统计: [('hello', 3), ('world', 2), ('python', 1), ('test', 1)]
Tokens to indices: ['hello', 'world', 'python', 'unknown_word', '<bos>'] → [4, 5, 6, 0, 2]
Indices to tokens: [0, 1, 2, 3, 4, 5] → ['<unk>', '<pad>', '<bos>', '<eos>', 'hello', 'world']

=== 示例2: 二维token列表 ===
_token_freqs: [('I', 2), ('love', 2), ('Python', 2), ('is', 2), ('python', 1), ('awesome', 1), ('coding', 1), ('fun', 1)]
idx_to_token:  ['<unk>', '<pad>']
token_to_idx: {'<unk>': 0, '<pad>': 1}
new_

[Discussions](https://discuss.d2l.ai/t/2094)
